# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd
import zipfile as zp

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [22]:
# your code here
zf = zp.ZipFile('Orders.zip','r')
#zf.namelist() # lists contents in 'Orders.zip', which is one item: Orders.csv

for filename in [ 'Orders.csv' ]: # the start of something dynamic but for now we're just reading this one file and putting it into orders
    try:
        orders = pd.read_csv(zf.open(filename))
    except KeyError:
        print('ERROR: Did not find %s in zip file' % filename)
    else:
        print(filename, ':')
    print
    
orders.head()

Orders.csv :


,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [48]:
# Sub problem 1
customer_spent = orders.groupby(['CustomerID','Country']).agg({'amount_spent':'sum'})
display(customer_spent.shape)

# Sub problem 2
vip_customers = customer_spent[customer_spent['amount_spent'] >= customer_spent['amount_spent'].quantile(0.95)]
preferred_customers = customer_spent[(customer_spent['amount_spent'] >= customer_spent['amount_spent'].quantile(0.75)) & (customer_spent['amount_spent'] < customer_spent['amount_spent'].quantile(0.95))]
display(vip_customers.shape)
display(preferred_customers.shape)
# check! -sizes are as expected

# Sub problem 3
# I guess we can use the above in checks to create a new column in our customer_spent or orders table - to answer questions we'll need country too but can tweak to capture this in our customer_spent if we want to use that smaller df

# lets reset index to make customerid a column
customer_spent.reset_index

#Attempt1: 
#lets just capture the values of the qantiles
global vip_amount
global pref_amount
vip_amount = customer_spent['amount_spent'].quantile(0.95)
pref_amount = customer_spent['amount_spent'].quantile(0.75)

def magic(x):
    if x >= vip_amount:
        return 'VIP'
    elif x >= pref_amount:
        return 'Preferred'
    else:
        return 'Ya Basic'

#pokemon['Name'] = pokemon['Name'].apply(strip_before_mega)
customer_spent['customer_type'] = customer_spent['amount_spent'].apply(magic)
customer_spent.reset_index()

# tweaked sub problem 1 to bring in country...

(4347, 1)

(218, 1)

(869, 1)

,CustomerID,Country,amount_spent,customer_type
0,12346,United Kingdom,77183.60,VIP
1,12347,Iceland,4310.00,Preferred
2,12348,Finland,1797.24,Preferred
3,12349,Italy,1757.55,Preferred
4,12350,Norway,334.40,Ya Basic
...,...,...,...,...
4342,18280,United Kingdom,180.60,Ya Basic
4343,18281,United Kingdom,80.82,Ya Basic
4344,18282,United Kingdom,178.05,Ya Basic
4345,18283,United Kingdom,2094.88,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [65]:
# your code here
cus = customer_spent # i'm lazy

# list of number of VIPs for each country
country_and_vips = cus[cus['customer_type']=='VIP'].groupby(['Country']).agg({'amount_spent':'size'})    # 'size' = count of rows, seems better to use this than using pd.Series.nunique as that relies the column being unique to get the actual count rather than count distinct

country_and_vips.reset_index()
country_and_vips = country_and_vips.rename(columns={'amount_spent':'VIPs'})
country_and_vips.reset_index()

country_and_vips[country_and_vips['VIPs'] == country_and_vips['VIPs'].max()]

,VIPs
Country,
United Kingdom,178


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [69]:
# your code here
# so copy pasta from above and adjust our VIP filter....and column names if I'm feeling energetic enough...

cus = customer_spent # i'm lazy

# list of number of VIPs for each country
country_and_vips = cus[cus['customer_type'].isin(['VIP','Preferred'])].groupby(['Country']).agg({'amount_spent':'size'})    # 'size' = count of rows, seems better to use this than using pd.Series.nunique as that relies the column being unique to get the actual count rather than count distinct

country_and_vips.reset_index()
country_and_vips = country_and_vips.rename(columns={'amount_spent':'VIPs'})
country_and_vips.reset_index()

# check! expect sum to this df to equal length of our vip_customers+preferred_customers, which it does \o/
#country_and_vips.sum()

country_and_vips[country_and_vips['VIPs'] == country_and_vips['VIPs'].max()]

,VIPs
Country,
United Kingdom,934


In [ ]:
# hmmm, might try and tree map this up...